In [ ]:
import os, pickle

import numpy as np
import pandas as pd

from scipy.io import loadmat
from scipy.optimize import curve_fit

import StaticWindow as sw

import matplotlib.pyplot as plt
import seaborn as sns

# Load Data and Tidy

In [ ]:
fname = 'dataout_333.mat.cor.mat'
data = loadmat(fname)['ch2']

In [ ]:
# Visualise Loaded Trace
fig, ax = plt.subplots(figsize=(6, 4), dpi=600)
all_x = np.linspace(0, 10, 10000000)
ax.plot(all_x, data * 1E9)

ax.set_xlabel("Time / s", weight='bold')
ax.set_ylabel("Current / nA", weight='bold')

ax.tick_params(width=1.5)
for axis in ['top', 'right', 'bottom', 'left']:
    ax.spines[axis].set_linewidth(1.5)

# fig.savefig('Expt Trace.png')

In [ ]:
# Remove sloping feature and flip the y axis
data = data[600000:] * -1
fig, ax = plt.subplots()
ax.plot(data)

In [ ]:
# Convert y axis to pico amps
data_scaled = data * 1E12
fig, ax = plt.subplots()
h = ax.hist(data_scaled, bins=147)

# Event Detection

Three repeat runs on experimental data are performed. Each repeat utilised a different window size and step size.

## Analyse Background 1

In [ ]:
background_results_1 = sw.analyse_background([data_scaled], 1000, 100)
means_1, bin_mids_1, bin_counts_1, gauss_params_1, norm_factor_1 = background_results_1

In [ ]:
fig, ax = plt.subplots()
ax.bar(bin_mids_1, bin_counts_1 / norm_factor_1, np.diff(bin_mids_1)[0])

x = np.linspace(bin_mids_1[0], bin_mids_1[-1], 100)
y = sw.gauss(x, *gauss_params_1)

ax.plot(x, y / norm_factor_1, color='red')

In [ ]:
threshold = 0.9999
gauss_params_norm_1 = gauss_params_1.copy()
gauss_params_norm_1[0] = gauss_params_norm_1[0] / norm_factor_1
thresh_val_1 = sw.limit_finder(gauss_params_norm_1, means_1.min(), means_1.max(), threshold)
thresh_val_1

## Event Search 1

In [ ]:
window_size = 1000
step_size = 100

In [ ]:
evt_locs1 = sw.event_search([data_scaled], thresh_val_1, window_size, step_size)
evt_locs1

## Analyse Background 2

In [ ]:
background_results_2 = sw.analyse_background([data_scaled], 100, 10)
means_2, bin_mids_2, bin_counts_2, gauss_params_2, norm_factor_2 = background_results_2

In [ ]:
fig, ax = plt.subplots()
ax.bar(bin_mids_2, bin_counts_2 / norm_factor_2, np.diff(bin_mids_2)[0])

x = np.linspace(bin_mids_2[0], bin_mids_2[-1], 100)
y = sw.gauss(x, *gauss_params_2)

ax.plot(x, y / norm_factor_2, color='red')

In [ ]:
threshold = 0.9999
gauss_params_norm_2 = gauss_params_2.copy()
gauss_params_norm_2[0] = gauss_params_norm_2[0] / norm_factor_2
thresh_val_2 = sw.limit_finder(gauss_params_norm_2, means_2.min(), means_2.max(), threshold)
thresh_val_2

## Event Search 2

In [ ]:
window_size = 100
step_size = 10

In [ ]:
evt_locs2 = sw.event_search([data_scaled], thresh_val_2, window_size, step_size)
evt_locs2

## Analyse Background 3

In [ ]:
background_results_3 = sw.analyse_background([data_scaled], 500, 50)
means_3, bin_mids_3, bin_counts_3, gauss_params_3, norm_factor_3 = background_results_3

In [ ]:
fig, ax = plt.subplots()
ax.bar(bin_mids_3, bin_counts_3 / norm_factor_3, np.diff(bin_mids_3)[0])

x = np.linspace(bin_mids_3[0], bin_mids_3[-1], 100)
y = sw.gauss(x, *gauss_params_3)

ax.plot(x, y / norm_factor_3, color='red')

In [ ]:
threshold = 0.9999
gauss_params_norm_3 = gauss_params_3.copy()
gauss_params_norm_3[0] = gauss_params_norm_3[0] / norm_factor_3
thresh_val_3 = sw.limit_finder(gauss_params_norm_3, means_3.min(), means_3.max(), threshold)
thresh_val_3

## Event Search 3

In [ ]:
window_size = 500
step_size = 50

In [ ]:
evt_locs3 = sw.event_search([data_scaled], thresh_val_3, window_size, step_size)
evt_locs3

# Figures

In [ ]:
# Plot a comparison of the three different background analyses

props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

fig, axs = plt.subplots(1, 3, figsize=(6, 3), dpi=600, sharey=True, sharex=True)

axs[0].bar(bin_mids_2, bin_counts_2 / norm_factor_2, np.diff(bin_mids_2)[0])
x2 = np.linspace(bin_mids_2[0], bin_mids_2[-1], 100)
y = sw.gauss(x2, *gauss_params_2)
axs[0].plot(x2, y / norm_factor_2, color='red')
text_str = "Gaussian\nMean: {:.2f}\nStd: {:.2f}".format(gauss_params_2[1], gauss_params_2[2])
axs[0].text(0.4, 0.95, text_str, transform=axs[0].transAxes,
            verticalalignment='top', bbox=props)

axs[1].bar(bin_mids_3, bin_counts_3 / norm_factor_3, np.diff(bin_mids_3)[0])
# x = np.linspace(bin_mids_3[0], bin_mids_3[-1], 100)
y = sw.gauss(x2, *gauss_params_3)
axs[1].plot(x2, y / norm_factor_3, color='red')
text_str = "Gaussian\nMean: {:.2f}\nStd: {:.2f}".format(gauss_params_3[1], gauss_params_3[2])
axs[1].text(0.4, 0.95, text_str, transform=axs[1].transAxes,
            verticalalignment='top', bbox=props)

axs[2].bar(bin_mids_1, bin_counts_1 / norm_factor_1, np.diff(bin_mids_1)[0])
# x = np.linspace(bin_mids_1[0], bin_mids_1[-1], 100)
y = sw.gauss(x2, *gauss_params_1)
axs[2].plot(x2, y / norm_factor_1, color='red')
text_str = "Gaussian\nMean: {:.2f}\nStd: {:.2f}".format(gauss_params_1[1], gauss_params_1[2])
axs[2].text(0.4, 0.95, text_str, transform=axs[2].transAxes,
            verticalalignment='top', bbox=props)

labels = ['(a)', '(b)', '(c)']
axs[0].set_ylabel("Normalised Counts", weight='bold')
for i, ax in enumerate(axs):
    ax.set_xlabel("Current / pA", weight='bold')
    ax.annotate(labels[i], xy=(0, 0), xytext=(-5, 0.6), weight='bold')
    ax.tick_params(width=1.5)
    for axis in ['top', 'right', 'bottom', 'left']:
        ax.spines[axis].set_linewidth(1.5)
    
    
fig.tight_layout()
# fig.savefig("Expt Background.png")

## Tidy events
Fix overlapping events

In [ ]:
def tidy_events(evt_locs):
    new_evt_locs = []
    
    clashes = 0
    i = 0
    while i < (len(evt_locs) - 1):
        evt = evt_locs[i]
        
        fut_evt = evt_locs[i+1]
        if evt[1] > fut_evt[0]:
            # print('CLASH')
            clashes += 1
            new_evt = [evt[0], fut_evt[1]]
            new_evt_locs.append(new_evt)
            i += 2
        else:
            new_evt_locs.append(evt)
            i += 1
            
    if clashes == 0:
        return evt_locs
    else:
        return tidy_events(new_evt_locs)        

In [ ]:
evt_locs1_tidy = tidy_events(evt_locs1[0])
evt_locs2_tidy = tidy_events(evt_locs2[0])
evt_locs3_tidy = tidy_events(evt_locs3[0])

## Analyse Events

In [ ]:
# Calculate the height and duration of every detected event

def calc_height_and_dur(evt_locs, trace):
    durs = []
    heights = []
    
    for start, end in evt_locs:
        dur = (end - start) / 1000000 * 1000
        height = np.max(trace[start:end])
        durs.append(dur)
        heights.append(height)
    
    return np.array(durs), np.array(heights)
    
durs1, heights1 = calc_height_and_dur(evt_locs1_tidy, data_scaled)
durs2, heights2 = calc_height_and_dur(evt_locs2_tidy, data_scaled)
durs3, heights3 = calc_height_and_dur(evt_locs3_tidy, data_scaled)

In [ ]:
# Combine data in DataFrame
columns = ['window_size', 'duration', 'height']

ws1 = np.array(['1000' for x in durs1])
ws2 = np.array(['100' for x in durs2])
ws3 = np.array(['500' for x in durs3])

df1 = pd.DataFrame(dict(zip(columns, [ws1, durs1, heights1])))
df2 = pd.DataFrame(dict(zip(columns, [ws2, durs2, heights2])))
df3 = pd.DataFrame(dict(zip(columns, [ws3, durs3, heights3])))

df = pd.concat((df2, df3, df1))
df.describe()

In [ ]:
# Plot scatter plot and distributions of event duration and event height.
g = sns.JointGrid(height=5)
sns.scatterplot(data=df, x='duration', y='height', hue='window_size', ax=g.ax_joint, alpha=0.5)
sns.histplot(data=df, x='duration', hue='window_size', ax=g.ax_marg_x, bins=60)
sns.histplot(data=df, y='height', hue='window_size', ax=g.ax_marg_y, bins=60)

g.ax_joint.get_legend().set_title("Window Size")
g.ax_marg_x.get_legend().remove()
g.ax_marg_y.get_legend().remove()
g.ax_marg_x.set(ylim=(0, 50))
g.ax_marg_y.set(xlim=(0, 50))

g.ax_joint.set_xlabel('Event Duration / ms', weight='bold')
g.ax_joint.set_ylabel('Event Height / pA', weight='bold')

g.ax_joint.tick_params(width=1.5)
for axis in ['top', 'right', 'bottom', 'left']:
    g.ax_joint.spines[axis].set_linewidth(1.5)
    
g.figure.tight_layout()
g.figure.set_dpi(600)

g.figure.savefig('Transloc Evts.png', dpi=600)

## Example Events

In [ ]:
pad = 5000
x = np.linspace(0, len(data_scaled), len(data_scaled))
plot_x = x / 1000000 * 1000

fig, axs = plt.subplots(3, 1, figsize=(6, 4), dpi=600, sharey=True)
start, end = evt_locs1[0][3] # Hand selected event
axs[0].plot(plot_x[start-pad : end+pad], data_scaled[start-pad : end+pad])
axs[0].plot(plot_x[start : end], data_scaled[start : end], color='red')

start, end = evt_locs1[0][0] # Hand selected event
axs[1].plot(plot_x[start-pad : end+pad], data_scaled[start-pad : end+pad])
axs[1].plot(plot_x[start : end], data_scaled[start : end], color='red')

start, end = evt_locs1[0][-1] # Hand selected event
axs[2].plot(plot_x[start-pad : end+pad], data_scaled[start-pad : end+pad])
axs[2].plot(plot_x[start : end], data_scaled[start : end], color='red')

labels = ['(a)', '(b)', '(c)']
for i, ax in enumerate(axs):
    ax.set_ylabel("Current / pA", weight='bold')
    ax.tick_params(width=1.5)
    for axis in ['top', 'right', 'bottom', 'left']:
        ax.spines[axis].set_linewidth(1.5)
    ax.annotate(labels[i], xy=(-0.17, 1), xycoords='axes fraction', weight='bold')
axs[2].set_xlabel("Time / ms", weight='bold')

fig.tight_layout()

# fig.savefig("Example RPS Evts.png")